In [ ]:
%load_ext pycodestyle_magic
%pycodestyle_on
#%flake8_on

# Race condition in Python

Incrementing a simple value in one single Thread is easy an produces no errors. 
As soon as you have multiple Threads working with a shared variable, problems can arise. **Race conditions** can occure, this means the value of the shared variable is modified in an erronous way.

Let's have a look at the following example. There is a class `Counter` with a method `increment`. This method does nothing else than incrementing the value by 1.
Multiple threads (here two) share an instance of the object `Counter` and invoke the `increment` method. Both threads do this one million times. Thus the final value of the counter should be 2 millions.

***But after execution you will see that this is not the case*** Why?

In [ ]:
from threading import Thread


class Counter(object):
    def __init__(self):
        self.value = 0

    def increment(self):
        self.value += 1


c = Counter()


def go():
    for i in range(1000000):
        c.increment()


# Run two threads that increment the counter:
t1 = Thread(target=go)
t1.start()
t2 = Thread(target=go)
t2.start()
t1.join()
t2.join()
print(c.value)

## The race condition problem

We incremented 2,000,000 times, but that’s not what we got. The problem is that self.value += 1 actually takes three distinct steps:

1. Getting the attribute,
2. incrementing it,
3. then setting the attribute.

If two threads call `increment()` on the same object around the same time, the following series steps may happen:

```
Thread 1: Get self.value, which happens to be 13.
Thread 2: Get self.value, which happens to be 13.
Thread 1: Increment 13 to 14.
Thread 1: Set self.value to 14.
Thread 2: Increment 13 to 14.
Thread 1: Set self.value to 14.
```
An increment was lost due to a race condition.


### **LOCKS** one possible solution
But this will be seen later.... let's first have a look on the **critical section**


In [ ]:
from threading import Thread
from threading import Lock


class Counter(object):
    def __init__(self):
        self.value = 0
        self.lock = Lock()

    def increment(self):
        with self.lock:
            self.value += 1


c = Counter()


def go():
    for i in range(1000000):
        c.increment()


# Run two threads that increment the counter:
t1 = Thread(target=go)
t1.start()
t2 = Thread(target=go)
t2.start()
t1.join()
t2.join()
print(c.value)

Resource: https://codewithoutrules.com/2017/08/16/concurrency-python/

## General form of a threaded program in Python

In [ ]:
from threading import Thread

S = 4       # number of "sessions"
# S = 400000  # number of "sessions"
N = 6       # the number of processes
x = 0       # shared ressource


def my_process():
    for i in range(S):
        global x
        # print("Session ", i)

        # non-critical section
        # ...
        # entry protocol
        # critical section
        x = x + 1
        # exit protocol


threads = []
for j in range(N):
    thread = Thread(target=my_process)
    threads.append(thread)
    thread.start()

for thread in threads:  # iterates over the threads
    thread.join()       # waits until the thread has finished work

print(x)

## Critical section
The following code snippets show tries to cope with the critical section (CS)
### First attempt
*see JR book p. 46*
 

In [ ]:
from threading import Thread


x = 0       # shared ressource
turn = 0    # who is in CS ?
N = 3       # number of iterations


def process1():
    global turn, x
    print("P1 started ")
    for j in range(N):
        print(f"P1({j}) started ")
        # non-critical section
        print(f"P1({j}) outside CS ")
        # entry protocol
        while(turn != 0):
            pass  # does nothing
        # critical section
        print(f"P1({j}) in CS ")
        x = x + 3
        # exit protocol
        turn = 1


def process2():
    global turn, x
    print("P2 started ")
    for j in range(N):
        print(f"P2({j}) started ")
        # non-critical section
        print(f"P2({j}) outside CS ")
        # entry protocol
        while (turn != 1):
            pass  # does nothing
        # critical section
        print(f"P2({j}) in CS ")
        x = x + 3
        # exit protocol
        turn = 0


# start the threads
threads = []
threads.append(Thread(target=process1))
threads.append(Thread(target=process2))
threads[0].start()
threads[1].start()

# wait until all processes have finished
for thread in threads:
    thread.join()

print(x)

| | | |
|-|--------|-|
| mutual exclusion | OK | turn can't be 1 and 2 at the same time |
| absence of dead/livelock | OK | if turn can be set to something else than 1 or 2 (e.g. hacker, buggy program, wrong initialization) |
| unnecessary delay | NOK | e.g. if turn=1 and P2 is waiting for a user input in the non-critical zone, then P1 must wait unnecessary to enter its CS |
| fairness | OK | both CS alternate always |



### Second attempt

In [7]:
from threading import Thread


x = 0       # shared ressource
N = 3       # number of iterations

c1 = False      # process1 sets to 1 in CS
c2 = False      # process2 sets to 1 in CS


def process1():
    global c1, c2, x
    print("P1 started ")
    for j in range(N):
        print(f"P1({j}) started ")
        # non-critical section
        print(f"P1({j}) outside CS ")
        # entry protocol
        while (c2):     #while(c2 == True):
            pass  # does nothing
        c1 = True
        # critical section
        print(f"P1({j}) in CS ")
        x = x + 3
        # exit protocol
        c1 = False


def process2():
    global c1, c2, x
    print("P2 started ")
    for j in range(N):
        print(f"P2({j}) started ")
        # non-critical section
        print(f"P2({j}) outside CS ")
        # entry protocol
        while (c1):     # while (c1 == True):
            pass  # does nothing
        c2 = True
        # critical section
        print(f"P2({j}) in CS ")
        x = x + 3
        # exit protocol
        c2 = False


# start the threads
threads = []
threads.append(Thread(target=process1))
threads.append(Thread(target=process2))
threads[0].start()
threads[1].start()

# wait until all processes have finished
for thread in threads:
    thread.join()

print(x)

P1 started P2 started 
P1(0) started 
P1(0) outside CS 
P2(0) started 
P2(0) outside CS 
P2(0) in CS 
P2(1) started 
P2(1) outside CS 
P2(1) in CS 
P2(2) started 
P2(2) outside CS 
P2(2) in CS 

P1(0) in CS 
P1(1) started 
P1(1) outside CS 
P1(1) in CS 
P1(2) started 
P1(2) outside CS 
P1(2) in CS 
18


### Third attempt

In [11]:
from threading import Thread


x = 0       # shared ressource
N = 3       # number of iterations

c1 = False      # process1 sets to 1 in CS
c2 = False      # process2 sets to 1 in CS


def process1():
    global c1, c2, x
    print("P1 started ")
    for j in range(N):
        print(f"P1({j}) started ")
        # non-critical section
        print(f"P1({j}) outside CS ")
        # entry protocol
        c1 = True
        while (c2): pass  # while c2 == True wait
        # critical section
        print(f"P1({j}) in CS ")
        x = x + 3
        # exit protocol
        c1 = False


def process2():
    global c1, c2, x
    print("P2 started ")
    for j in range(N):
        print(f"P2({j}) started ")
        # non-critical section
        print(f"P2({j}) outside CS ")
        # entry protocol
        c2 = True
        while (c1): pass  # while c1 == True wait
        # critical section
        print(f"P2({j}) in CS ")
        x = x + 3
        # exit protocol
        c2 = False


# start the threads
threads = []
threads.append(Thread(target=process1))
threads.append(Thread(target=process2))
threads[0].start()
threads[1].start()

# wait until all processes have finished
for thread in threads:
    thread.join()

print(x)

P1 started 
P1(0) started 
P1(0) outside CS 
P1(0) in CS 
P1(1) started 
P1(1) outside CS 
P1(1) in CS 
P1(2) started 
P1(2) outside CS 
P1(2) in CS 
P2 started 
P2(0) started 
P2(0) outside CS 
P2(0) in CS 
P2(1) started 
P2(1) outside CS 
P2(1) in CS 
P2(2) started 
P2(2) outside CS 
P2(2) in CS 
18


### Fourth attempt

In [12]:
from threading import Thread


x = 0       # shared ressource
N = 3       # number of iterations

c1 = False      # process1 sets to 1 in CS
c2 = False      # process2 sets to 1 in CS


def process1():
    global c1, c2, x
    print("P1 started ")
    for j in range(N):
        print(f"P1({j}) started ")
        # non-critical section
        print(f"P1({j}) outside CS ")
        # entry protocol
        c1 = True
        while(c2):
            c1 = False
            c1 = True
        # critical section
        print(f"P1({j}) in CS ")
        x = x + 3
        # exit protocol
        c1 = False


def process2():
    global c1, c2, x
    print("P2 started ")
    for j in range(N):
        print(f"P2({j}) started ")
        # non-critical section
        print(f"P2({j}) outside CS ")
        # entry protocol
        c2 = True
        while (c1):
            c2 = False
            c2 = True
        # critical section
        print(f"P2({j}) in CS ")
        x = x + 3
        # exit protocol
        c2 = False


# start the threads
threads = []
threads.append(Thread(target=process1))
threads.append(Thread(target=process2))
threads[0].start()
threads[1].start()

# wait until all processes have finished
for thread in threads:
    thread.join()

print(x)

P1 started 
P1(0) started 
P1(0) outside CS 
P1(0) in CS 
P1(1) started 
P1(1) outside CS 
P1(1) in CS 
P1(2) started 
P1(2) outside CS 
P1(2) in CS 
P2 started 
P2(0) started 
P2(0) outside CS 
P2(0) in CS 
P2(1) started 
P2(1) outside CS 
P2(1) in CS 
P2(2) started 
P2(2) outside CS 
P2(2) in CS 
18


### Fifth attempt
This is the Dekker algorithm

In [ ]:
from threading import Thread


x = 0       # shared ressource
N = 3       # number of iterations

enter1 = False      # process1 demands to enter CS
enter2 = False      # process2 demands to enter CS
turn = 1            # who is in CS ?


def process1():
    global enter1, enter2, turn, x
    print("P1 started ")
    for j in range(N):
        print(f"P1({j}) started ")
        # non-critical section
        print(f"P1({j}) outside CS ")
        # entry protocol
        enter1 = True
        while (enter2):
            if (turn == 2):
                enter1 = False
                while (turn == 2):
                    pass
                enter1 = True
        # critical section
        print(f"P1({j}) in CS ")
        x = x + 3
        # exit protocol
        enter1 = False
        turn = 2


def process2():
    global enter1, enter2, turn, x
    print("P2 started ")
    for j in range(N):
        print(f"P2({j}) started ")
        # non-critical section
        print(f"P2({j}) outside CS ")
        # entry protocol
        enter2 = True
        while (enter1):
            if (turn == 1):
                enter2 = False
                while (turn == 1):
                    pass
                enter2 = True
        # critical section
        print(f"P2({j}) in CS ")
        x = x + 3
        # exit protocol
        enter2 = False
        turn = 1


# start the threads
threads = []
threads.append(Thread(target=process1))
threads.append(Thread(target=process2))
threads[0].start()
threads[1].start()

# wait until all processes have finished
for thread in threads:
    thread.join()

print(x)